In [1]:
import edge_tts
import asyncio
import util   
import config
import json
from IPython.display import Audio
from edge_tts_vc_func import RVCSpeakers


2023-08-10 02:36:40 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-08-10 02:36:41 | INFO | faiss.loader | Loading faiss with AVX2 support.
2023-08-10 02:36:41 | INFO | faiss.loader | Successfully loaded faiss with AVX2 support.


In [2]:
rvc = RVCSpeakers(hubert_model_path='./hubert_base.pt', rvc_config_file='./multi_config.json')

2023-08-10 02:36:42 | INFO | fairseq.tasks.hubert_pretraining | current directory is /media/gpt4-pdf-chatbot-langchain/RVC-Speakers
2023-08-10 02:36:42 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2023-08-10 02:36:42 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1, 'dropout

Loading model: yiqing
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: bob
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: arianagrande
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: syz
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: lulu
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: maomao
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Models loaded: 6


In [3]:
import librosa


async def vc_func(
        rvc: RVCSpeakers, audio_file_path, model_index, pitch_adjust, f0_method, 
        feat_ratio, filter_radius, rms_mix_rate, resample_option 
):
 
    if model_index is None:
        return (None, 'Please select a model.')
      
    audio, sr = librosa.load(audio_file_path, sr=None)  # sr=None 表示保持原始采样率

    return rvc.vc_func(
        (sr, audio),
        model_index,
        pitch_adjust,
        f0_method,
        feat_ratio,
        filter_radius,
        rms_mix_rate,
        resample_option
    )



In [4]:

loaded_models=[
        '%s - %s' % (
            m['metadata'].get('source', 'Unknown'),
            m['metadata'].get('name')
        )
        for m in rvc.loaded_models
]
loaded_models

['Bilibili - 一清清清',
 'ALL - Bob Sponge',
 'ALL - Ariana Grande',
 'ALL - Stefanie Sun',
 '露露 - 露露',
 '毛毛 - 毛毛']

In [5]:

model_index = loaded_models.index('毛毛 - 毛毛')
model_index

5

In [12]:
 

#变调(整数, 半音数量, 升八度12降八度-12)
f0_up_key=-12
f0_method =["pm", "harvest", "crepe", "rmvpe"]

#检索特征占比
index_rate = -12
#>=3则使用对harvest音高识别的结果使用中值滤波，数值为滤波半径，使用可以削弱哑音
filter_radius=2
#输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络
rms_mix_rate=1
#后处理重采样至最终采样率，0为不进行重采样
resample_rate=0
 
 

In [13]:
audio_file_path = 'bark_generation.wav'

out_sr, output_audio = await vc_func(rvc,audio_file_path,model_index,f0_up_key,f0_method[3],
                                index_rate, filter_radius, rms_mix_rate, resample_rate)


-12
npy: 0.026424407958984375s, f0: 0.13139820098876953s, infer: 0.15525555610656738s


In [14]:
from scipy.io.wavfile import write as write_wav

# save audio to disk
write_wav(f"test.wav", out_sr, output_audio) 

# play text in notebook
Audio(output_audio, rate=out_sr)